# Tiling Tool Example 

This document goes through code examples of using the tiling tool, alongside an explanation on its input and what each does.  

In [ ]:
# this example only 
import geemap
import ipywidgets as widgets

# default 
import ee
from tiler import projTiler # assuming its called tiler.py and is in the same directory

# ee.Authenticate()
# ee.Initialize(project="jameswilliamchamberlain")

In [ ]:
# example default polygon (Aprox. Exeter area)
polygon = ee.Geometry.Polygon([[[-3.548069, 50.735695], [-3.566608, 50.735586], [-3.563862, 50.708961], [-3.514938, 50.676229], [-3.483009, 50.670789], [-3.442154, 50.678839], [-3.394089, 50.733413], [-3.427734, 50.74797], [-3.488503, 50.753618], [-3.548584, 50.748187], [-3.548069, 50.735695]]])

m = geemap.Map(center=[50.735243, -3.533585], zoom=15)
m.add_draw_control()
m.addLayer(polygon, {'color': 'grey', 'fillColor': '00000000'}, 'Input Polygon')

# add button for update default input polygon to last drawn

def update_polygon(b):
    """Updates Input Polygon"""

    if m.draw_last_feature is not None:
        polygon = m.draw_last_feature.geometry()

    m.addLayer(polygon, {'color': 'grey', 'fillColor': '00000000'}, 'Input Polygon')

def draw_2025(b):
    """Draws RGB Composite for 2025 Senteinl-2"""

    image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterBounds(polygon) \
        .filterDate("2025-01-01", "2025-12-31") \
        .sort("CLOUD_COVER") \
        .first() \
        .clip(polygon)
    
    rgb_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}
    m.addLayer(image, rgb_params, "Sentinel-2 2025 RGB")


show_tiles_btn = widgets.Button(description="Update Input Polygon", position="bottomright")
show_tiles_btn.on_click(update_polygon)
m.add_widget(show_tiles_btn, position="bottomright")

show_2025_btn = widgets.Button(description="Show 2025 Sentinel-2 RGB", position="bottomright")
show_2025_btn.on_click(draw_2025)
m.add_widget(show_2025_btn, position="bottomright")

draw_2025(None)

m

Map(center=[50.735243, -3.533585], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

# Default Input Paramaters  

`polygon`: The polygon to tile (A single polygon to tile)

`projection`: The selected GEE porjection at the selected band resolution - it is recommended to set the lowest resolution band that you intended to use here and base your grid resolution on that, as othewise this band may not perfeclty algin with the generated tiles.        

`width_px`: In Pixels the width (tiles are assumed to have same width/height)

(self, polygon, projection, width_px, assetId_prefix=None, shpfile_max_polygons=71**2, export_fn=export_tiles_to_gee, identifier_name_fn=lambda col, row, lon_min, lon_max, lat_min, lat_max: f"col{col}_row{row}_lonmin{lon_min}__lonmax{lon_max}_latmin{lat_min}__latmax{lat_max}"):

In [ ]:
# Band 1 is selected meanign that width_px=1 would mean 60m pixels, width_px=2 would be 120m pixels etc.
# 6 means is 6x60m = 360m x 360m pixels for each tile
# it is best to select the highest resolution *Band as then there will be no issues with lower resolution bands not aligning with the pixel grid

# Ideally select the lowest resolution band that you will be using to avoid issues with alignment within your data.
#       As Sentienl-1 and Sentienl-2 have multiple resolutions for different bands and 
#           higher resolution bands fit perfeclty within their lower resolution counterparts however 
#           created tiles at lower resolutions may not align perfectly with higher resolution bands after generating tiles.
#           As such if you were say working with all Sentienl-2 bands and you wanted a resolution of 60m after, B1 or any other 60m band would be a far better choice for the input

projection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(polygon) \
    .first() \
    .select("B1") \
    .projection()

# polygon - already set feel free to adjust in the GEEMAP UI above two buttons have been provided to update polygon and show 2025 RGB (B4, B3, B2 for S2)

print(projection.getInfo())

{'type': 'Projection', 'crs': 'EPSG:32630', 'transform': [60, 0, 399960, 0, -60, 5700000]}


## Visulaisation of Output
This is just an example of showing generated tiles, geemap may lag if alot of tiles are generated so its recommended only to create a smaller polygon (but up to you)

In [16]:
tiler = projTiler(polygon, projection, assetId_prefix=None, width_px=6, shpfile_max_polygons=71**2)
tiles = tiler.tile_lst

for i, tile_collection in enumerate(tiles):
    m.addLayer(tile_collection.style(**{'color': 'red', 'fillColor': '00000000'}), {}, f'tile collection {i}')

m

Max width px per shapefile: 426.0
5041 max polygons per shapefile
6 tile width px
Tiles created: 1
Tiles created: 666
No assetId_prefix provided, skipping export.


Map(center=[50.735243, -3.533585], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

## Identifier Columns

`identifier_name_fn`: function to set the unique identifier name for each tile. 

`identifier_col`: unique identifier column name 


### Unique identifier Column (`identifer`) 
this is where the unique identifier is stored for each tile and is by default set by a lambda function

`identifier_name_fn`: function to set the unique identifier name for each tile. 
`identifier_col`: a string column name that can be used to set the identifier column name across all tiles 

In [ ]:
# defualt identifier prefix is set below in the projTiler class instantiation
identifier_name_fn=lambda col, row, lon_min, lon_max, lat_min, lat_max: f"col{col}_row{row}_lonmin{lon_min}__lonmax{lon_max}_latmin{lat_min}__latmax{lat_max}"
identifier_col="unique_identifier_column" # default is just `identifier` 

projTiler(polygon, projection, width_px=6, identifier_name_fn=identifier_name_fn, identifier_col=identifier_col).tile_lst[0].getInfo()['columns']

Max width px per shapefile: 71.0
5041 max polygons per shapefile
1 tile width px
Tiles created: 9
Tiles created: 2444
Tiles created: 4576
Tiles created: 318
Tiles created: 4658
Tiles created: 5041
Tiles created: 672
Tiles created: 1140
Tiles created: 3320
Tiles created: 73
No assetId_prefix provided, skipping export.


{'col': 'Integer',
 'row': 'Integer',
 'system:index': 'String',
 'unique_identifier_column': 'String'}

### (Additional) Column and Row (`col` and `row`)

These columns have been added to each tile as this may be useful metadata later on for any processing you many want to do, its way easier to collect this data during this process now rather than later as it already needs to be converted to a format that can be represented in columns and rows as how a rendering software would need to represent the columns and rows of the data rather than their longitude and latitude representations. As such selecting the rows or columns makes it much easier to slice.

In [18]:
# 2 additional Columns, `col` and `row` 
tiles[0].getInfo()

# sorting by col/row and selecting first tiles col/row
first_col_tile = tiles[0].filter(ee.Filter.eq('col', tiles[0].first().get('col')))
first_row_tile = tiles[0].filter(ee.Filter.eq('row', tiles[0].first().get('row')))

# display first col tiles
m.addLayer(first_col_tile.style(**{'color': 'blue', 'fillColor': '00000000'}), {}, 'Selected Column tiles')
m.addLayer(first_row_tile.style(**{'color': 'green', 'fillColor': '00000000'}), {}, 'Selected Row tiles')

In [19]:
m

Map(bottom=352623.0, center=[50.72189478059448, -3.4373474121093754], controls=(WidgetControl(options=['positi…

## Saving & Exporting the tiles 

`assetId_prefix`: this is the prefix for the location that you want to export to, default is `None` but should be set unless you decided to treat `projTiler` as a variable to later extract the tiles (stored in `projTiler` as `self.tiles`)

`export_fn`: this is a function that must take 2 positional arguments `tiles` and `assetId_prefix`:

&emsp;   `tiles` - A list of ee.FeatureCollection where each collection represents 1 shapefile to save

&emsp;   `assetId_prefix` - The file location and prefix to filename


The export_fn is an input to this so that alternative locations can be set, for example in a case where you do not want to export to GEE, the default and an example of saving locally by changing the function can be seen in the next two cells: 

### Default Export 

In [20]:
# set assetId_prefix and upload tiles to your GEE assets
assetId_prefix = "projects/jameswilliamchamberlain/assets/shape_file_export_test/chunk2_" # assetId prefix file location and name prefix 
raise("Set these values before running the upload") # Comment this out once you have set the values

TypeError: exceptions must derive from BaseException

In [ ]:
# Output Location Defualt is this function here (is included as the default argument):
# this is set under as an input to the projTiler class 
from tiler import export_tiles_to_gee

# this is set by default so does not need to be set but is shown below for clarity

projTiler(polygon, projection, assetId_prefix=assetId_prefix, width_px=3, export_fn=export_tiles_to_gee) # B1 60m - so 3x60m = 180m x 180m tiles

# Below shows `export_tiles_to_gee` function that is the default set in projTiler, this has only been tested for small amounts of tile uploads so if tasks fail you may need to adjust this. 

# import time
# def export_tiles_to_gee(tiles, assetId_prefix, i_offset=0, sleep_time=5):
#     if assetId_prefix is None:
#         print("No assetId_prefix provided, skipping export.")
#         return
#     # upload each tile (and wait to avoid failures)
#     for i, tile in enumerate(tiles):
#         # chunk_name = f"chunk_{i+1}"
#         task = ee.batch.Export.table.toAsset(
#             collection=tile,
#             description=f"export_{i_offset+i+1}",
#             assetId=f"{assetId_prefix}{i_offset+i+1}"
#         )
#         task.start()
#         time.sleep(sleep_time)

### Saving Locally 

This is just an example of how to replace the function for storing in alternative locations or other personal adjustments 

In [ ]:
# example of saving locally 
def export_tiles_to_local(tiles, assetId_prefix, i_offset=0):
    import os
    import time
    import geopandas as gpd
    
    # split into folder and filename
    folder = os.path.dirname(assetId_prefix)
    if not os.path.exists(folder):
        os.makedirs(folder)
    filename = os.path.basename(assetId_prefix)

    # makedir 
    if not os.path.exists(assetId_prefix):
        os.makedirs(assetId_prefix)

    # save 
    for i, tile in enumerate(tiles):
        geojson_str = tile.getInfo()
        gdf = gpd.GeoDataFrame.from_features(geojson_str['features'])
        shapefile_path = os.path.join(folder, f"{filename}_tile_{i_offset + i + 1}.shp")
        gdf.to_file(shapefile_path)

        print(f"Saved tile {i_offset + i + 1} to {shapefile_path}")
        time.sleep(1)

projTiler(polygon, projection, assetId_prefix="exported_tiles/example_local_", width_px=1, export_fn=export_tiles_to_local)